In [244]:
%matplotlib inline

from IPython.display import display
from ipywidgets import widgets
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict as od
import random
import re
import seaborn as sns

#pd.set_option('html', False)
np.set_printoptions(threshold=np.nan)
sns.set()

In [137]:
path = '../btc-data/BTC_Trades_raw.csv'
date_parse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

data = pd.read_csv(path)

data.TradeData = data.TradeData.map(lambda x: float(re.sub('[^0-9,.]', '', x)))
data.TradeID = data.TradeID.map(lambda x: float(re.sub('[^0-9,.]', '', x)))

data.rename(columns = {data.columns[0]: 'Amount', data.columns[1]: 'Price'}, inplace = True)
data.insert(0, 'Trade ID', data.index)

data['Date'] = data['Date'].map(date_parse)

data.set_index('Date', inplace = True)
data.sort_values(by = 'Trade ID', inplace = True)

data.to_csv(path_or_buf='../btc-data/BTC_Trades_clean.csv')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [258]:
def generate_ob_frame():
    path = '../btc-data/BTC_OB_raw.csv'
    date_parse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

    data = pd.read_csv(path, sep='",', nrows=5000)

    data.rename(columns = {data.columns[0]: 'LOB Data', 
                           data.columns[1]: 'ID', 
                           data.columns[2]: 'Date'}, inplace = True)

    data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('asks', 'asks ', x))
    data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('bids', 'bids ', x))
    data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('[^0-9,.,asks ,bids ,]', '', x))
    data['ID'] = data['ID'].map(lambda x: re.sub('\D', '', x))
    data['Date'] = data['Date'].map(lambda x: re.sub('[^0-9,/,:, ]', '', x))

    data['Date'] = data['Date'].map(date_parse)

    bids = data['LOB Data'].map(lambda x: re.split(',', re.sub('bids ', '', re.sub('^asks [0-9,.]*', '', x))))
    asks = data['LOB Data'].map(lambda x: re.split(',', re.sub('asks ', '', re.sub('bids [0-9,.]*', '', x)[:-1])))

    data.insert(1, 'Bids', bids)
    data.insert(1, 'Asks', asks)

    data['Bids'] = data['Bids'].map(lambda x: list(zip(x[::2], x[1::2])))
    data['Asks'] = data['Asks'].map(lambda x: list(zip(x[::2], x[1::2])))

    data.set_index('Date', inplace = True)
    data.drop('LOB Data', axis = 1, inplace = True)

    data = data[['ID','Asks', 'Bids']]
    data['ID'] = data['ID'].map(lambda x: int(x))
    data.sort_values(by = 'ID', inplace = True)

#data.to_csv(path_or_buf='../btc-data/BTC_OB_clean_test.csv')
%time generate_ob_frame()

/Users/dlucsanszky/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


CPU times: user 3.22 s, sys: 13.3 s, total: 16.6 s
Wall time: 20.5 s


In [257]:
askside = []
bidside = []

print(data.tail())

for i in range(len(data)):
    for ask in data.ix[i, 'Asks']:
        if float(ask[0]) in askside:
            if float(ask[1]) == 0:
                askside.remove(float(ask[0]))
            else:
                index = askside.index(ask[0])
                #update volume
                askside[index][1] += ask[1]
        elif len(askside) >= 50:
            if float(ask[1]) != 0:
                if max(askside) > ask[0]:
                    askside.remove(max(askside))
                    askside.append((float(ask[0]), float(ask[1])))
        else:
            if float(ask[1]) != 0:
                askside.append((float(ask[0]), float(ask[1])))
    
    for bid in data.ix[i, 'Bids']:
        if float(bid[0]) in bidside:
            if float(bid[1]) == 0:
                bidside.remove(float(bid[0]))
            else:
                index = bidside.index(bid[0])
                #update volume
                bidside[index][1] += bid[1]
        elif len(bidside) >= 50:
            if float(bid[1]) != 0:
                if min(bidside) > bid[0]:
                    bidside.remove(min(bidside))
                    bidside.append((float(bid[0]), float(bid[1])))
        else:
            if float(bid[1]) != 0:
                bidside.append((float(bid[0]), float(bid[1])))
        
print(list(askside))
print(len(askside))
print(list(bidside))
print(len(bidside))

                          ID  \
Date                           
2016-04-24 15:21:26  6530914   
2016-04-24 15:21:27  6530915   
2016-04-24 15:21:28  6530916   
2016-04-24 15:21:29  6530917   
2016-04-24 15:21:30  6530918   

                                                                  Asks  \
Date                                                                     
2016-04-24 15:21:26                      [(454.23000000, 48.75231050)]   
2016-04-24 15:21:27  [(452.11000000, 0), (455.80000000, 202.83000000)]   
2016-04-24 15:21:28  [(451.97000000, 0.25482676), (453.19000000, 0)...   
2016-04-24 15:21:29                                                 []   
2016-04-24 15:21:30                       [(453.19000000, 0.55000000)]   

                                                                 Bids  
Date                                                                   
2016-04-24 15:21:26                               [(443.92000000, 0)]  
2016-04-24 15:21:27  [(448.39000000, 15.

In [242]:
data.Bids = data['Bids'].map(float, re.split(',', re.sub('[^0-9,.]', '', b)))
    
data.head()

TypeError: float() argument must be a string or a number, not 'list'